In [0]:
#!kill -9 -1

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
os.chdir('/content/gdrive/My Drive/s2vt')

In [0]:
!apt install caffe-cpu


Reading package lists... Done
Building dependency tree       
Reading state information... Done
caffe-cpu is already the newest version (1.0.0-6).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [0]:
cd 


/root


In [0]:
pwd

'/root'

caffe installations

In [0]:
with open('/etc/apt/sources.list') as f:
  txt = f.read()
with open('/etc/apt/sources.list.backup', 'w') as f:
  f.write(txt)
with open('/etc/apt/sources.list', 'w') as f:
  f.write(txt.replace('# deb-src','deb-src'))

In [0]:
!apt update
!apt build-dep caffe-cuda
!apt install g++-5

In [0]:
!wget https://dl.bintray.com/boostorg/release/1.67.0/source/boost_1_67_0.tar.bz2
!tar --bzip2 -xf boost_1_67_0.tar.bz2
#!tar xjvf ~/boost_1_67_0.tar.bz2

In [0]:
!update-alternatives --remove-all gcc 
!update-alternatives --remove-all g++

!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-5 20
!update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-5 20

!update-alternatives --install /usr/bin/cc cc /usr/bin/gcc 30
!update-alternatives --set cc /usr/bin/gcc

!update-alternatives --install /usr/bin/c++ c++ /usr/bin/g++ 30
!update-alternatives --set c++ /usr/bin/g++

In [0]:
import sys 
print(sys.version)

3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]


In [0]:
!cd boost_1_67_0 && ./bootstrap.sh --exec-prefix=/usr/local --with-libraries=system,filesystem,regex,thread,python \
--with-python-version=3.6.7 --with-python-root=/usr
!cd boost_1_67_0 && ./b2 install

In [0]:
!git clone https://github.com/BVLC/caffe.git && cd caffe 

In [0]:
with open('caffe/Makefile.config.example') as f:
  config = f.read()
  
comment = ['CUDA_DIR := /usr/local/cuda', 
           'BLAS := open']
uncomment = ['# CUDA_DIR := /usr', 
             '# BLAS := atlas', 
             '# OPENCV_VERSION := 3', '# WITH_PYTHON_LAYER := 1'] #
replace = [('INCLUDE_DIRS := $(PYTHON_INCLUDE) /usr/local/include',
            'INCLUDE_DIRS := $(PYTHON_INCLUDE) /usr/local/include /usr/include/hdf5/serial /usr/local/lib/python2.7/dist-packages/numpy/core/include/'), 
           ('LIBRARY_DIRS := $(PYTHON_LIB) /usr/local/lib /usr/lib',
            'LIBRARY_DIRS := $(PYTHON_LIB) /usr/local/lib /usr/lib /usr/lib/x86_64-linux-gnu/hdf5/serial')]

for c in uncomment:
  config = config.replace(c, c[2:])
for c in comment:
  config = config.replace(c, '# '+c)
for c1,c2 in replace:
  config = config.replace(c1, c2)
  
with open('caffe/Makefile.config', 'w') as f:
  f.write(config)

In [0]:
with open('caffe/Makefile') as f:
  mfile = f.read()
  
with open('caffe/Makefile.backup', 'w') as f:
  f.write(mfile)
  
with open('caffe/Makefile', 'w') as f:
  f.write(mfile.replace('-isystem','-I'))

In [0]:
with open('/usr/include/x86_64-linux-gnu/c++/5/bits/c++config.h') as f:
  txt = f.read()
with open('/usr/include/x86_64-linux-gnu/c++/5/bits/c++config.h', 'w') as f:
  f.write(txt.replace('/* #undef _GLIBCXX_USE_C99_MATH */',
                      '/* #undef _GLIBCXX_USE_C99_MATH */\n#define  _GLIBCXX_USE_C99_MATH  1'))

In [0]:
!cd caffe && make -j8 && make pycaffe && make test -j8 && make distribute


PROTOC src/caffe/proto/caffe.proto
NVCC src/caffe/util/math_functions.cu
NVCC src/caffe/solvers/nesterov_solver.cu
NVCC src/caffe/solvers/sgd_solver.cu
NVCC src/caffe/solvers/adadelta_solver.cu
make: /usr/bin/nvcc: Command not found
NVCC src/caffe/solvers/rmsprop_solver.cu
Makefile:604: recipe for target '.build_release/cuda/src/caffe/util/math_functions.o' failed
make: *** [.build_release/cuda/src/caffe/util/math_functions.o] Error 127
make: *** Waiting for unfinished jobs....
NVCC src/caffe/solvers/adam_solver.cu
make: /usr/bin/nvcc: Command not found
Makefile:604: recipe for target '.build_release/cuda/src/caffe/solvers/adadelta_solver.o' failed
make: *** [.build_release/cuda/src/caffe/solvers/adadelta_solver.o] Error 127
make: /usr/bin/nvcc: Command not found
NVCC src/caffe/util/im2col.cu
Makefile:604: recipe for target '.build_release/cuda/src/caffe/solvers/sgd_solver.o' failed
make: *** [.build_release/cuda/src/caffe/solvers/sgd_solver.o] Error 127
make: /usr/bin/nvcc: Command no

In [0]:
!echo /usr/local/lib >> /etc/ld.so.conf && ldconfig
!echo /content/caffe/distribute/lib >> /etc/ld.so.conf && ldconfig

In [0]:
pip install ipdb

In [0]:
import sys
sys.path.insert(0,'/content/gdrive/My Drive/s2vt/caffe/python')
#sys.path.insert(0,"/path/to/caffe/python")
#sys.path.insert(0, caffe_path[0])  
import caffe

In [0]:
cd

/root


In [0]:

import ipdb
import cv2
import numpy as np
import skimage


deploy = '/content/gdrive/My Drive/s2vt/VGG_ILSVRC_19_layers_deploy.prototxt'
model = '/content/gdrive/My Drive/s2vt/VGG_ILSVRC_19_layers.caffemodel'
mean = '/content/gdrive/My Drive/s2vt/ilsvrc_2012_mean.npy'

class CNN(object):
    import caffe
    def __init__(self, deploy=deploy, model=model, mean=mean, batch_size=10, width=227, height=227):

        self.deploy = deploy
        self.model = model
        self.mean = mean

        self.batch_size = batch_size
        self.net, self.transformer = self.get_net()
        self.net.blobs['data'].reshape(self.batch_size, 3, height, width)

        self.width = width
        self.height = height

    def get_net(self):
        caffe.set_mode_cpu()
        net = caffe.Net(self.deploy, self.model, caffe.TEST)

        transformer = caffe.io.Transformer({'data':net.blobs['data'].data.shape})
        transformer.set_transpose('data', (2,0,1))
        transformer.set_mean('data', np.load(self.mean).mean(1).mean(1))
        transformer.set_raw_scale('data', 255)
        transformer.set_channel_swap('data', (2,1,0))

        return net, transformer

    def get_features(self, image_list, layers='fc7', layer_sizes=[4096]):
        iter_until = len(image_list) + self.batch_size
        all_feats = np.zeros([len(image_list)] + layer_sizes)

        for start, end in zip(range(0, iter_until, self.batch_size), \
                              range(self.batch_size, iter_until, self.batch_size)):

            image_batch = image_list[start:end]

            caffe_in = np.zeros(np.array(image_batch.shape)[[0,3,1,2]], dtype=np.float32)

            for idx, in_ in enumerate(image_batch):
                caffe_in[idx] = self.transformer.preprocess('data', in_)

            out = self.net.forward_all(blobs=[layers], **{'data':caffe_in})
            feats = out[layers]

            all_feats[start:end] = feats

        return all_feats


In [0]:
import cv2
import os
import ipdb
import numpy as np
import pandas as pd
import skimage

#from cnn_util import *

In [0]:
def preprocess_frame(image, target_height=224, target_width=224):

    if len(image.shape) == 2:
        image = np.tile(image[:,:,None], 3)
    elif len(image.shape) == 4:
        image = image[:,:,:,0]

    image = skimage.img_as_float(image).astype(np.float32)
    height, width, rgb = image.shape
    if width == height:
        resized_image = cv2.resize(image, (target_height,target_width))

    elif height < width:
        resized_image = cv2.resize(image, (int(width * float(target_height)/height), target_width))
        cropping_length = int((resized_image.shape[1] - target_height) / 2)
        resized_image = resized_image[:,cropping_length:resized_image.shape[1] - cropping_length]

    else:
        resized_image = cv2.resize(image, (target_height, int(height * float(target_width) / width)))
        cropping_length = int((resized_image.shape[0] - target_width) / 2)
        resized_image = resized_image[cropping_length:resized_image.shape[0] - cropping_length,:]

    return cv2.resize(resized_image, (target_height, target_width))

In [0]:
def main():
    import caffe
    num_frames = 80

    vgg_deploy = '/content/gdrive/My Drive/s2vt/VGG_ILSVRC_19_layers_deploy.prototxt'
    vgg_model = '/content/gdrive/My Drive/s2vt/VGG_ILSVRC_19_layers.caffemodel'

    video_path = '/content/gdrive/My Drive/s2vt/youtube_videos'
    video_save_path = '/content/gdrive/My Drive/s2vt/youtube_feats'
    videos = os.listdir(video_path)
    videos = filter(lambda x: x.endswith('avi'), videos)

    cnn = CNN(model=vgg_model, deploy=vgg_deploy, width=224, height=224)

    for idx, video in enumerate(videos):
        print( idx+1, video)

        if os.path.exists( os.path.join(video_save_path, video) ):
            print ("Already processed ... ")
            continue

        video_fullpath = os.path.join(video_path, video)
        try:
            cap  = cv2.VideoCapture( video_fullpath )
        except:
            pass

        frame_count = 0
        frame_list = []

        while True:
            ret, frame = cap.read()
            if ret is False:
                break

            frame_list.append(frame)
            frame_count += 1

        frame_list = np.array(frame_list)

        if frame_count > 80:
            frame_indices = np.linspace(0, frame_count, num=num_frames, endpoint=False).astype(int)
            frame_list = frame_list[frame_indices]

        cropped_frame_list = np.array(list(map(lambda x: preprocess_frame(x), frame_list)))
        feats = cnn.get_features(cropped_frame_list)

        save_full_path = os.path.join(video_save_path, video + '.npy')
        np.save(save_full_path, feats)


In [0]:
import os
if __name__=="__main__":
    main()


1 0lh_UWF9ZP4_71_77.avi
2 0lh_UWF9ZP4_103_110.avi
3 0lh_UWF9ZP4_79_82.avi
4 0k1Ak8aTMVI_4_12.avi
5 _6OTzzK7t9Y_158_170.avi
6 0hyZ__3YhZc_598_603.avi
7 0IDJG0q9j_k_1_24.avi
8 0lh_UWF9ZP4_50_60.avi
9 0I4VKj4d0WI_24_40.avi
10 0lh_UWF9ZP4_38_46.avi
11 _txL575S_OA_13_23.avi
12 0lh_UWF9ZP4_27_31.avi
13 _QJbjKkfdcs_52_57.avi
14 0hyZ__3YhZc_279_283.avi
15 _O9kWD8nuRU_77_81.avi
16 0hyZ__3YhZc_352_356.avi
17 0lh_UWF9ZP4_21_26.avi
18 _xf24TYgbuY_14_20.avi
19 00jrXRMlZOY_0_10.avi
20 _6OTzzK7t9Y_73_78.avi
21 _UqnTFs1BLc_23_27.avi
22 _ZwwKOzpt2I_69_76.avi
23 _pBeOAMjGm8_1_18.avi
24 _pj2Nutu5v8_1_8.avi
25 -_hbPLsZvvo_323_328.avi
26 -_aaMGK6GGw_57_61.avi
27 _o1UXSxTjfo_68_80.avi
28 _9iG5Ge01PM_3_11.avi
29 _0nX-El-ySo_83_93.avi
30 _JVxurtGIhI_32_42.avi
31 _O9kWD8nuRU_45_49.avi
32 _1vy2HIN60A_32_40.avi
33 -_hbPLsZvvo_5_8.avi
34 _KMoC6ZdeXs_10_16.avi
35 -_hbPLsZvvo_49_55.avi
36 _7nP9z6T9m8_11_17.avi
37 _OBlgSz8sSM_15_32.avi
38 _O9kWD8nuRU_25_35.avi
39 3mbBbFH9fAg_21_29.avi
40 -_hbPLsZvvo_288_305.avi
41 3